Considering TCP/IP protocols stack, it is a five-layers stack, as shown below. The protocols in physical layer define the voltage, baudrate etc., while the protocols in data link define the data structure based on the physical layer. The Network Layer has protocols we are familiar with like IP protocol and ARP protocol.
```
-------------------------
|       Application     |
-------------------------
|       Transport       |
-------------------------
|       Network         |
-------------------------
|       Data Link       |
-------------------------
|       Physical        |
-------------------------
```
In embedded system, protocols like ISP, I2C, CAN etc. work at physical layer. The Mavlink protocol focused here works at data link Layer as we can see the word 'Link' in its name. Here the `MAVLink` refers in particular to `MAVLink 2.0` is anlsysed. There are some important features different from previous version, for more details check the [official docs](https://mavlink.io/en/guide/mavlink_2.html). The Mavlink protocol defines some microservices, which function as network layer and transport layer.

### 1 Data Packet Analysing
Below is a screenshot of panes of Wireshark, which equipped dissector generated by `mavgenerate.py`. It's shown as a layer structure. I filtered the data using `udp.port==14580` which is the port opened by PX4 to communicated with external developer APIs. In the *upper* pane of Wireshark, current data transferred on the `localhost` were shown by the order of the time they were captured. In the protocol column, the MAVLink were shown in some rows, while other rows were shown in black, which are ICMP messages to tell the PX4 there is no external API calling that is working and accessible. In the *middle* pane, the selected data on the upper pane. the dissected data was shown in layer structure. And, In the *bottom* pane, the raw data is shown in hex and ascii format.
![picture of data dissected by Wireshark](data_dissected_screenshot.png)

Here, the dissected data in the middle pane was pasted below to explain the **MAVLink** data structure.

```
Frame 245: 82 bytes on wire (656 bits), 82 bytes captured (656 bits) on interface 0
Ethernet II, Src: 00:00:00_00:00:00 (00:00:00:00:00:00), Dst: 00:00:00_00:00:00 (00:00:00:00:00:00)
Internet Protocol Version 4, Src: localhost (127.0.0.1), Dst: localhost (127.0.0.1)
User Datagram Protocol, Src Port: 14570, Dst Port: 14550
MAVLink Protocol (40)
    Header
        Magic value / version: 0xfd
        Payload length: 28
        Incompatibility flag: 0
        Compatibility flag: 0
        Packet sequence: 4
        System id: 0x01
        Component id: 0x01
        Message id: 0x000020
    Payload: LOCAL_POSITION_NED (32)
        time_boot_ms (uint32): 388692
        x (float): -0.013624
        y (float): -0.0130454
        z (float): -0.0302121
        vx (float): 0.0118521
        vy (float): -0.0282792
        vz (float): 0.00425027
    Message CRC: 0x0b0e
```

The first line is a summary information. The next three lines show the protocols from the physical layer to the transport layer. The fifth line show the MAVLink Protocol data, which has size of 40 bytes. The original data was shown as hex data as below.
```
0000   fd 1c 00 00 04 01 01 20 00 00 54 ee 05 00 34 37   ý...... ..Tî..47
0010   5f bc 3a bc 55 bc 6d 7f f7 bc 57 2f 42 3c da a9   _¼:¼U¼m.÷¼W/B<Ú©
0020   e7 bc d7 45 8b 3b 0e 0b                           ç¼×E.;..

```
The data frame in MAVLink is shown below diagram, from the official docs, for more info see[[2]](https://mavlink.io/en/guide/serialization.html)
![MAVLink v2.0 packert frame](packet_mavlink_v2.jpg)
According to above data frame, the hex data can be explained as 
```
fd -- STX, version number, specifically version 2.0
1c -- Payload length, 28 in decimal digit
00 -- incompatibility flag, 0, not incompatible 
00 -- Compatibility flag, 0, not set for compatiblility
04 -- Packet sequence, 4
... 
```
By doing so, we can get the result achieved by wireshark.

### 2 Data Packeting Analysing
In above section, the data structure of MAVLink data is described. In this section, the way to get the data frame will be analysed, i.e. the details of the meaning and generation of each field. Here, frame and packet have same meaning.

2.1 STX
STX stands for *start-of-text*, which can be used to identify the start of beginning. **0xff** means MAVLink version 1.0, while **0xfe** means MAVLink version 2.0

2.2 LEN
The Length of the payload, since the payload is limited in 255 bytes, the LEN field has size of one byte. 

2.3 INC FLAGS
This field include some flags which could has special meanings and if it was set to **1** the receiver will ignore the total frame if it can't understand it. Currently, there is only one bit used.

2.4 CMP FLAGS
This field include some flags which indicate some special features which can be ignored if receiver can't understand it and parse other field.

2.5 SEQ
SEQ is the serial number of current Frame. It will increase by one for any new packet.

2.6 SYS ID
It is used to identify the vehicle.

2.7 COMP ID
It is used to identify the component. A vehicle may have several device that is able to transmit MAVLink data.

2.8 MSG ID
Since there are many different type of data on a vehicle needed to transmit, which will have size obviously big and have different updating frequencies. So data with different usage or different sensors will be transmit in different frame and `MSG ID`is used to differentiate them.  The data in different type of message can be extended by append `<extension>` field in XML file.

2.9 PAYLOAD
It is data need to transmit. It's size is variable and depends on MSG ID. The data will be reordered and zero-filled bytes at the end will be truncated.

2.10 CHECKSUM
CHECKSUM is field of checksum of `PAYLOAD` field data. According to the code in the [MAVLink](https://github.com/mavlink/c_library_v2/blob/master/mavlink_helpers.h) pasted below, the checksum is calculated through the CRC_EXTRA, which is get by CRC-16-CCITT on the format type, and the data field described above.

```
    msg->checksum = crc_calculate(&buf[1], header_len-1);
    crc_accumulate_buffer(&msg->checksum, _MAV_PAYLOAD(msg), msg->len);
    crc_accumulate(crc_extra, &msg->checksum);
    mavlink_ck_a(msg) = (uint8_t)(msg->checksum & 0xFF);
    mavlink_ck_b(msg) = (uint8_t)(msg->checksum >> 8);
```

#TODO
reading the C implement of MAVLink protocol to see the management of links.

#### References
[1] Design Document https://docs.google.com/document/d/1XtbD0ORNkhZ8eKrsbSIZNLyg9sFRXMXbsR2mp37KbIg/edit?usp=sharing

[2] Frame https://mavlink.io/en/guide/serialization.html